In [1]:
import os
import numpy as np                                          # arrays and matrix math
import pandas as pd                                         # DataFrames

from sklearn import metrics                                 # measures to check our models
from sklearn.model_selection import train_test_split        # train and test split
from sklearn.ensemble import RandomForestRegressor          # random forest method
from sklearn.cluster import KMeans
import joblib


#modules
import pymongo
import dns
from datetime import date
from config import mongo_uri
import pprint
import json

In [2]:
#connect to mongo db
client = pymongo.MongoClient(mongo_uri)

In [3]:
top_tracks = client.spotify['user-data'].find({'name':'Henry Byoun'})[0]['top_50_tracks']

In [5]:
danceability = []
energy = []
speechiness = []
instrumentalness = []
liveness = []
valence = []
tempo = []



for i in range(len(top_tracks)):
    danceability.append(top_tracks[i]['audio_features'][0]['danceability'])
    energy.append(top_tracks[i]['audio_features'][0]['energy'])   
    speechiness.append(top_tracks[i]['audio_features'][0]['speechiness'])   
    instrumentalness.append(top_tracks[i]['audio_features'][0]['instrumentalness']) 
    liveness.append(top_tracks[i]['audio_features'][0]['liveness']) 
    valence.append(top_tracks[i]['audio_features'][0]['valence'])  
    tempo.append(top_tracks[i]['audio_features'][0]['tempo'])  
 

In [6]:
dance = np.mean(danceability)
en = np.mean(energy)
speech = np.mean(speechiness)
inst = np.mean(instrumentalness)
live = np.mean(liveness)
val = np.mean(valence)
temp = np.mean(tempo)

In [11]:
data = [(dance,en,speech,inst,live,val,temp)]

In [12]:
data

[(0.5416799999999999,
  0.618714,
  0.07999200000000001,
  0.08528957499999999,
  0.15297200000000002,
  0.4505759999999999,
  122.93969999999999)]

In [20]:
'''
The entries below have to follow the order. 
['danceability','energy','speechiness','instrumentalness','liveness','valence','tempo']
'''


filename = 'finalized_model.sav'
loaded_model = joblib.load(filename)
predicted_cluster = int(loaded_model.predict(data))

In [14]:
df = pd.read_csv('clustered_data.csv')

In [15]:
df_selected = df[df['cluster_num'] == predicted_cluster]

In [16]:
rec = df_selected.sample(5)

In [17]:
rec

,track,artist,uri,danceability,energy,speechiness,instrumentalness,liveness,valence,tempo,chorus_hit,cluster_num
39944,Eu Me Tornei um Mutante,Zumbis Do Espaço,spotify:track:2CqLn9YHk9tRDZmg0oms7U,0.452,0.917,0.0667,0.000019,0.0901,0.415,103.627,106.09566,450
15881,Witch of Endor - Instrumental,Moondog,spotify:track:51W1ugFbwJGA74yCd5iHhB,0.292,0.418,0.0352,0.170000,0.1440,0.182,103.624,100.71820,450
26798,Take Me To The River,Talking Heads,spotify:track:1oV1tu8utgHQjLJsEK9sVl,0.660,0.485,0.0258,0.010200,0.0585,0.829,99.828,100.85550,450
11106,Overstepping,Belmont,spotify:track:79XKPlY5fagGd2iXhDPrEs,0.525,0.989,0.0757,0.001680,0.3440,0.471,102.569,99.68481,450
18234,Magic Moon (Clair De Lune),The Rays,spotify:track:4lD1obJ7EeHuKwpKJHIPIr,0.485,0.351,0.0253,0.000074,0.1620,0.292,102.672,99.01630,450


In [18]:
rec['uri']=rec['uri'].str.split(':').str.get(2)

In [19]:
rec

,track,artist,uri,danceability,energy,speechiness,instrumentalness,liveness,valence,tempo,chorus_hit,cluster_num
39944,Eu Me Tornei um Mutante,Zumbis Do Espaço,2CqLn9YHk9tRDZmg0oms7U,0.452,0.917,0.0667,0.000019,0.0901,0.415,103.627,106.09566,450
15881,Witch of Endor - Instrumental,Moondog,51W1ugFbwJGA74yCd5iHhB,0.292,0.418,0.0352,0.170000,0.1440,0.182,103.624,100.71820,450
26798,Take Me To The River,Talking Heads,1oV1tu8utgHQjLJsEK9sVl,0.660,0.485,0.0258,0.010200,0.0585,0.829,99.828,100.85550,450
11106,Overstepping,Belmont,79XKPlY5fagGd2iXhDPrEs,0.525,0.989,0.0757,0.001680,0.3440,0.471,102.569,99.68481,450
18234,Magic Moon (Clair De Lune),The Rays,4lD1obJ7EeHuKwpKJHIPIr,0.485,0.351,0.0253,0.000074,0.1620,0.292,102.672,99.01630,450


In [32]:
recs = {}
rec_songs = [f'https://open.spotify.com/embed/track/{i}' for i in rec['uri']]

In [34]:
recs['song_recs'] = rec_songs

In [35]:
recs

{'song_recs': ['https://open.spotify.com/embed/track/6UhrIkR4sDh8LpoWdUVzt1',
  'https://open.spotify.com/embed/track/7qpP87krX9Hc2pR3if51pC',
  'https://open.spotify.com/embed/track/2ogF71NvoPUwJmqyKzEHqP',
  'https://open.spotify.com/embed/track/6gGie940lliTLLt85tZPp7',
  'https://open.spotify.com/embed/track/0B0FsdzH1KyPL1kFFj4taf']}